### Формула Maximum Value Extraction из пулов ликвидности Constant Product AMM

#### 1. Класс пула ликвидности и алгоритм AB продажи-покупки

Для начала необходимо реализовать класс пула ликвидности, в котором будет хранится информация об изначальном резерве каждого токена. Также внутри класса реализован алгоритм AB продажи-покупки, где на основе проданного кол-ва монет одного токена вычисляется купленное кол-во монет другого токена.

In [54]:
class LP: # liquidity pool
    fees = 0.003 # 0.3 % of fees returned to LP providers after a customer's transaction 
    
    def __init__(self, x, y):
        self.x = x # initial amount of token A in LP
        self.y = y # initial amount of token B in LP
        self.k = x * y # constant product
        
    def trade_return(self, dx):
        '''Calculating how much (dy) of token B is received after selling dx of token A'''
        dx *= (1+LP.fees) # taking the fees into account
        dy = (self.y * dx) / (self.x + dx)
        
        #self.x += dx # goes into the pool (sold by a customer)
        #self.y -= dy # goes out of the pool (bought by a customer)
        return dy

#### 2. Маршрут транзакции A->B->C->A и расчет полученной прибыли

Далее реализована цепочка транзакций для определенного кол-ва пулов, через которые последовательно происходит обмен монет токенов. Также вычисляется прибыль, получаемая при определенном кол-ве проданных монет первоначального токена А.

In [ ]:
def final_trade_return(initial_a, pools):
    '''Calculating the amount of token A received after completing the transaction chain'''
    out = initial_a # initial deposit of token A
    for lp in pools:
        out = lp.trade_return(out)
    return out

def absolute_income(initial_a, pools, sign=1):
    '''Calculating the relative income:
           ai = final_a - initial_a'''
    final_a = final_trade_return(initial_a, pools)
    ai = final_a - initial_a
    return sign * ai # sign '-' is used as we minimize the function

#### 3. Реализация оптимизационного метода Нелдера-Мида для функции одной переменной

Так как функция прибыли от изначального числа проданных токенов унимодальна (эмпирический вывод), используется оптимизационный метод первого порядка (метод Нелдера-Мида) для нахождения максимума функции.

In [95]:
from math import *

# The Nelder-Mead Algorithm
def nelder_mead(m, beta, gamma, epsilon, n = 1): 
    """
    m - edge length,
    beta - expansion coefficient, gamma - contraction coefficion,
    epsilon - critical error,
    n - number of dimensions (1 by default)
    """
    # Initial Simplex Point
    x0 = 1

    # Changes
    delta_1 = m * (sqrt(n + 1) - 1) / (n * sqrt(2))
    delta_2 = m * (sqrt(n + 1) + n - 1) / (n * sqrt(2))

    # The rest of vertices
    x1 = x0 + delta_1
    x2 = x0 + delta_2

    iteration = 0
    while (True): # while the error is bigger than the critical value, do the cycle

        iteration += 1

        # A dictionary of (x, f(x)) pairs
        dict = {x0:f(x0), x1:f(x1), x2:f(x2)} 

        # Sorting the pairs according to the function values in ascending order
        points = sorted(dict.items(), key = lambda x: x[1])

        l = points[0][0] # an argument of the lowest function value
        s = points[1][0] # an argument of the second-highest function value
        h = points[2][0] # an argument of the highest function value

        # Boolean parameters denoting if the actions were already performed
        reflection = False 
        expansion = False
        contraction = False
        
        x_c = (s + l) / 2 # the centroid of all points except H
        x_refl = 2 * x_c - h

        # Reflection
        if ((not reflection) and f(x_refl) < f(h)):
            h = x_refl
            reflection = True

        # Expansion
        if (reflection and f(x_refl) < f(l)):
            x_refl = x_c + beta * (h - x_c)

            if (f(x_refl) < f(h)):
                h = x_refl
                expansion = True

        # Contraction
        if ((not reflection) and (not expansion) and (f(s) < f(x_refl)) and (f(x_refl) < f(h))):
            x_refl = x_c + gamma * (h - x_c)

            if (f(x_refl) < f(h)):
                h = x_refl
                contraction = True

        # Reduction
        if ((not reflection) and (not expansion) and (not contraction)):
            x_red = l if f(l) < f(x_refl) else x_refl # corresponding to the lowest function value amongst l, s, h, and x_refl

            h = x_red + 0.5 * (h - x_red)
            s = x_red + 0.5 * (s - x_red)
            l = x_red + 0.5 * (l - x_red)

        # Updating the points
        x0 = h
        x1 = s
        x2 = l
        x_c = (x0 + x1 + x2) / 3 # the centroid of the whole polyhedron

        # Error
        sigma = sqrt(((f(x0) - f(x_c))**2 + (f(x1) - f(x_c))**2 + (f(x2) - f(x_c))**2) / 3)
        
        #print('Iteration #', iteration-1, ': ', l, ', ', f(l))

        if (sigma < epsilon):
            break

    return l

#### 4. Вывод наиболее выгодного числа токенов А для продажи на основе входных пулов ликвидности

In [107]:
from functools import partial

def find_optimal_a(pools):
    f = partial(absolute_income, pools=pools, sign=-1) # -1 * absolute_income (max = -min, for Nelder-Mead)

    optimal_a = nelder_mead(m=1, beta=2.0, gamma=0.5, epsilon=0.0001)
    print(f"Оптимальным количеством монет токена А для продажи равно: {int(optimal_a)}")

#### 5. Пример

In [108]:
ab = LP(3753139396, 166740188573)
bc = LP(724520588560, 766050680304)
cd = LP(10457920653, 1051487855)
de = LP(1722571966294, 2846977754550)
ea = LP(22496742244741, 4310194783973)

pools = [ab, bc, cd, de, ea]

find_optimal_a(pools)

Оптимальным количеством монет токена А для продажи равно: 47717739


#### 6. Ввод и вывод

И теперь организуем I/O поток для нахождения максимума с произвольными пулами

In [ ]:
n = int(input('Пожалуйста, введите кол-во пулов ликвидности в цепочке транзакций:'))
pools = []
for _ in range(n):
    balances = input('Пожалуйста, введите балансы двух токенов пула через пробел:').split()
    b1, b2 = int(balances[0]), int(balances[1])
    pools.append(LP(b1, b2))

print()
find_optimal_a(pools)